In [1]:
import numpy as np
import pandas as pd
import pandas.io.sql as pdsql
import pickle
import sys
from IPython.display import clear_output

import psycopg2

In [2]:
# Read the dropoff_count table and the day_info table
conn = psycopg2.connect("dbname='nyc_taxi' user='postgres' host='localhost' password='organon'")
#cdf = pdsql.read_sql("SELECT * FROM dropoff_count_ct_yellow", conn, coerce_float=True, params=None)
cdf = pdsql.read_sql("SELECT * FROM dropoff_count_ct_green", conn, coerce_float=True, params=None)
day_info = pdsql.read_sql("SELECT * FROM day_info", conn, coerce_float=True, params=None)
conn.close()

In [3]:
# Add the rows where count is 0
gid_append = []
doy_append = []
hour_append = []

row = 0
for gid in range(1, 2167): # 195 for NTA, 2166 for CT
    for doy in range(1, 366):
        for hour in range(24):
            if row < len(cdf.index) and\
            cdf.dropoff_gid[row] == gid and\
            cdf.dropoff_doy[row] == doy and\
            cdf.dropoff_hour[row] == hour:
                row += 1
            else:
                gid_append.append(gid)
                doy_append.append(doy)
                hour_append.append(hour)
                
    clear_output()
    print("gid = {0} completed".format(gid))
    sys.stdout.flush()
        
cdf_append = pd.DataFrame({'dropoff_gid': gid_append, 'dropoff_doy': doy_append, 'dropoff_hour': hour_append},
                          columns=['dropoff_gid', 'dropoff_doy', 'dropoff_hour'])
cdf_append['count'] = pd.Series([0] * len(gid_append))
cdf = cdf.append(cdf_append, ignore_index=True)
cdf = cdf.sort_values(by=['dropoff_gid', 'dropoff_doy', 'dropoff_hour'])
cdf.index = range(len(cdf))

gid = 2166 completed


In [4]:
# Add new columns, namely day of the week and the temperature, precipitation, holiday info
dropoff_dow = (cdf['dropoff_doy'] + 2) % 7 # For 2014, Jan. 1 is Wednesday
cdf['dropoff_dow'] = dropoff_dow

for col_name in {'temperature', 'precipitation', 'holiday'}:
    col = day_info[col_name][cdf['dropoff_doy'] - 1]
    col.index = range(len(col))
    cdf[col_name] = col

# Raw dataframe extracted from the original dataset ready for further process
cdf_raw = cdf[['dropoff_doy', 'count', 'dropoff_gid', 'dropoff_dow', 'dropoff_hour', 'temperature', 'precipitation', 'holiday']]

In [5]:
#cdf_raw.to_pickle('./intermediate_files/dataframes_visualize_ct/data_raw_dropoff_ct_yellow.p')
cdf_raw.to_pickle('./intermediate_files/dataframes_visualize_ct/data_raw_dropoff_ct_green.p')